In [1]:
# Kütüphaneleri yüklüyoruz.
import cv2
import numpy as np

In [2]:
from ultralytics import YOLO
# Telefonu tespit eden modeli yüklüyoruz
model = YOLO("yolo11l.pt")

In [3]:
from ultralytics import YOLO
# İnsanın bedenindeki bölgeleri tespit eden modeli yüklüyoruz
model2 = YOLO("yolo11n-pose.pt") 

Bu fonkisyon ile görüntüdeki telefonları tespit ediyoruz. Sonrasında telefonun orta noktasını bulup bunları bir listeye ekliyoruz. Birden fazla telefon olabilir.

In [4]:
def telefontespiti(img):
    noktaList=[]

    rgb_img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results = model(rgb_img,verbose=False) 
    labels=results[0].names



    for i in range(len(results[0].boxes)):
        x1,y1,x2,y2=results[0].boxes.xyxy[i]
        score=results[0].boxes.conf[i]

        label=results[0].boxes.cls[i]
        x1,y1,x2,y2,score,label=int(x1),int(y1),int(x2),int(y2),float(score),int(label)

        name=labels[label]



        if score<0.5:
            continue
        if name=='cell phone':
            cx=int(x1/2+x2/2)
            cy=int(y1/2+y2/2)
            noktaList.append((cx,cy))
    
    return noktaList

Burada insanın konumunu bölge olarak buluyoruz. Önemli nokta burada insanın telefonla konuşma ihtimali olan olan baş bölgesini bölge olarak bulmak. O yüzden ilk önce insanın tamamını region1 adlı değişkene atadım. Sonrasında omuz hizasına kadar olan bölgenin asıl bölge olmasını sağlamak için pose estimation ile 5. ve 6. noktaları gerekli konum bilgisine atadım. Birden fazla insan olabileceği için bunları bir  listeye ekledim.

In [5]:
def insantespiti(img):
    regionList=[]
    rgb_img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results = model2(rgb_img,verbose=False) 
    
    for i in range(len(results[0].boxes)):
        x1,y1,x2,y2=results[0].boxes.xyxy[i]
        x1,y1,x2,y2=int(x1),int(y1),int(x2),int(y2)
        region1=np.array([(x1,y1),(x2,y1),(x2,y2),(x1,y2)])
        region1 = region1.reshape((-1,1,2))
        regionList.append(region1)
    
    
    for j,keypoints in enumerate(results[0].keypoints.xy):
        
        for  i,keypoint in enumerate(keypoints):
            
            x3,y3=keypoint
        
            x3,y3=int(x3),int(y3)
            if i==5:
                regionList[j][2][0][1]=y3
            if i==6:
                regionList[j][3][0][1]=y3
            
       
        
        
        
    return regionList

# Görsel İçin

Buradaki kod ile resimdeki kişilerin o an telefon ile konuşup konuşmadığını bulabilirsiniz. Buradaki telefonların olduğu noktalar ile insanların olduğu bölgeleri kıyaslayarak her bir noktanın her bir bölge içinde olup olmadığına bakıyoruz. Eğer o bölgenin içinde ise inside_region1 değişkenin 1 oluyor.

In [8]:
img=cv2.imread('img.jpg')
font = cv2.FONT_HERSHEY_SIMPLEX

liste1=telefontespiti(img)
liste2=insantespiti(img)


for region2 in liste2:
    for (x,y) in liste1:
        inside_region1=cv2.pointPolygonTest(region2,(x,y),False)
        
        if inside_region1>0:
            
            cv2.putText(img, 'Telefon Konusmasi',(region2[0][0][0], region2[0][0][1]-30), font, 5, (0,100,255), 5)
            cv2.polylines(img,[region2],True,(0,0,255),15)

cv2.imshow("kamera",img)

cv2.waitKey()
cv2.destroyAllWindows()

# Video İçin

In [11]:
kamera= cv2.VideoCapture('video.mp4')

font = cv2.FONT_HERSHEY_SIMPLEX


while True:

   
    
    ret,kare=kamera.read()
    if not ret:
        break
        
    liste1=telefontespiti(kare)
    liste2=insantespiti(kare)
    
    for region2 in liste2:
        for (x,y) in liste1:
            inside_region1=cv2.pointPolygonTest(region2,(x,y),False)

            if inside_region1>0:

                cv2.putText(kare, 'Telefon Konusmasi',(region2[0][0][0], region2[0][0][1]-30), font, 3, (255,0,0), 3)
                cv2.polylines(kare,[region2],True,(102,0,153),10)
    
  
    
    
    cv2.imshow("kamera",kare)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
kamera.release()
cv2.destroyAllWindows()